In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# TensorFlow와 Keras 임포트
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

# 1. 데이터셋 이해 및 문제 정의
# - 데이터셋: 패션 MNIST (Fashion MNIST)
# - 목표: 주어진 의류 이미지를 10개의 카테고리 중 하나로 분류하는 모델을 구축합니다.
# - 문제 유형: 다중 클래스 분류
# - 평가 지표: 정확도(Accuracy)

# 2. 데이터 로드 및 탐색적 데이터 분석(EDA)
# 2.1 데이터 로드
# 패션 MNIST 데이터셋 로드
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# 2.2 데이터 구조 확인
# 훈련 데이터와 테스트 데이터의 크기 출력
print("훈련 데이터 이미지 형태:", X_train.shape)
print("훈련 데이터 레이블 형태:", y_train.shape)
print("테스트 데이터 이미지 형태:", X_test.shape)
print("테스트 데이터 레이블 형태:", y_test.shape)

# 2.3 데이터 샘플 시각화
# 클래스 이름 정의
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# 첫 번째 이미지 시각화
plt.figure()
plt.imshow(X_train[0], cmap='gray')
plt.title(f"레이블: {y_train[0]} ({class_names[y_train[0]]})")
plt.colorbar()
plt.show()

# 여러 이미지 시각화
# 25개의 이미지 시각화
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(X_train[i], cmap='gray')
    plt.title(class_names[y_train[i]])
    plt.axis('off')
plt.tight_layout()
plt.show()

# 3. 데이터 전처리 및 준비
# 3.1 데이터 정규화
# 픽셀 값을 0~1 사이로 스케일링
X_train = X_train / 255.0
X_test = X_test / 255.0

# 3.2 데이터 형태 변환
# DNN을 위한 데이터 형태 변환 (2D 이미지 -> 1D 벡터)
X_train_dnn = X_train.reshape(-1, 28 * 28)
X_test_dnn = X_test.reshape(-1, 28 * 28)

# CNN을 위한 데이터 형태 변환 (채널 차원 추가)
X_train_cnn = X_train.reshape(-1, 28, 28, 1)
X_test_cnn = X_test.reshape(-1, 28, 28, 1)

# 3.3 레이블 원-핫 인코딩
y_train_encoded = to_categorical(y_train, 10)
y_test_encoded = to_categorical(y_test, 10)

# 4. 모델 선택 및 설계
# 4.1 DNN 모델 설계
from tensorflow.keras.optimizers import Adam

dnn_model = Sequential([
    Dense(512, activation='relu', input_shape=(784,)),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

# 4.2 CNN 모델 설계
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# 5. 모델 구현 및 컴파일
# 5.1 DNN 모델 컴파일
dnn_model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# 5.2 CNN 모델 컴파일
cnn_model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# 6. 모델 학습 및 검증
# 6.1 DNN 모델 학습
dnn_history = dnn_model.fit(
    X_train_dnn, y_train_encoded,
    epochs=20,
    batch_size=128,
    validation_split=0.2
)

# 6.2 CNN 모델 학습
cnn_history = cnn_model.fit(
    X_train_cnn, y_train_encoded,
    epochs=20,
    batch_size=128,
    validation_split=0.2
)

# 7. 모델 평가 및 개선
# 7.1 DNN 모델 평가
dnn_loss, dnn_accuracy = dnn_model.evaluate(X_test_dnn, y_test_encoded)
print(f"DNN 모델 테스트 정확도: {dnn_accuracy * 100:.2f}%")

# 7.2 CNN 모델 평가
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test_cnn, y_test_encoded)
print(f"CNN 모델 테스트 정확도: {cnn_accuracy * 100:.2f}%")

# 7.3 학습 결과 시각화
# DNN 모델 정확도 시각화
plt.figure()
plt.plot(dnn_history.history['accuracy'], label='훈련 정확도')
plt.plot(dnn_history.history['val_accuracy'], label='검증 정확도')
plt.title('DNN 모델 정확도')
plt.xlabel('에포크')
plt.ylabel('정확도')
plt.legend()
plt.show()

# CNN 모델 정확도 시각화
plt.figure()
plt.plot(cnn_history.history['accuracy'], label='훈련 정확도')
plt.plot(cnn_history.history['val_accuracy'], label='검증 정확도')
plt.title('CNN 모델 정확도')
plt.xlabel('에포크')
plt.ylabel('정확도')
plt.legend()
plt.show()

# 8. 모델 저장 및 배포
# 8.1 모델 저장
dnn_model.save('dnn_fashion_mnist.h5')
cnn_model.save('cnn_fashion_mnist.h5')

# 8.2 모델 로드
# DNN 모델 로드
loaded_dnn_model = load_model('dnn_fashion_mnist.h5')

# CNN 모델 로드
loaded_cnn_model = load_model('cnn_fashion_mnist.h5')

# 8.3 새로운 데이터로 예측
# 예시로 테스트 데이터의 첫 번째 이미지 사용
new_image = X_test_cnn[0].reshape(1, 28, 28, 1)
prediction = cnn_model.predict(new_image)
predicted_label = np.argmax(prediction)

print(f"예측된 레이블: {predicted_label} ({class_names[predicted_label]})")
print(f"실제 레이블: {y_test[0]} ({class_names[y_test[0]]})")

# 이미지 시각화
plt.figure()
plt.imshow(X_test_cnn[0].reshape(28,28), cmap='gray')
plt.title(f"실제 레이블: {class_names[y_test[0]]}, 예측된 레이블: {class_names[predicted_label]}")
plt.axis('off')
plt.show()